In [24]:
def merge_csvs_to_df(folder):
    import os
    import csv
    import pandas as pd

    rows = []

    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith('.csv'):
                path = os.path.join(root, file)
                with open(path, newline='', encoding='utf-8') as f:
                    reader = csv.DictReader(f)
                    for row in reader:
                        rows.append(row)

    df = pd.DataFrame(rows)
    # Ensure correct types for sorting
    df['doc_id'] = df['doc_id'].astype(str)
    df['entity1_offset'] = pd.to_numeric(df['entity1_offset'])
    df['entity2_offset'] = pd.to_numeric(df['entity2_offset'])
    # Sort by doc_id, then offsets
    df = df.sort_values(by=['doc_id', 'entity1_offset', 'entity2_offset']).reset_index(drop=True)
    # Remove duplicate rows
    df = df.drop_duplicates().reset_index(drop=True)

    return df

folder = "LLM_benchmarks/other_data/cdr/test"
merged_df_test = merge_csvs_to_df(folder)
print(merged_df_test.shape)  # Print shape of the merged DataFrame
print(merged_df_test.head()) # Print first few rows 
folder_train = "LLM_benchmarks/other_data/cdr/train"
merged_df_train = merge_csvs_to_df(folder_train)
print(merged_df_train.shape)  # Print shape of the merged DataFrame
folder_dev = "LLM_benchmarks/other_data/cdr/dev"
merged_df_dev = merge_csvs_to_df(folder_dev)
print(merged_df_dev.shape)  # Print shape of the merged DataFrame
print(merged_df_dev.head()) # Print first few rows 

(5518, 16)
     doc_id                                       passage_text entity1_id  \
0  10087562  Torsade de pointes ventricular tachycardia dur...    D004280   
1  10087562  Torsade de pointes ventricular tachycardia dur...    D004280   
2  10087562  Torsade de pointes ventricular tachycardia dur...    D004280   
3  10087562  Torsade de pointes ventricular tachycardia dur...    D004280   
4  10087562  Torsade de pointes ventricular tachycardia dur...    D004280   

                   entity1_text entity1_type entity1_ann_id  entity1_offset  \
0  ['Dubutamine', 'dobutamine']     Chemical              2              72   
1  ['Dubutamine', 'dobutamine']     Chemical              2              72   
2  ['Dubutamine', 'dobutamine']     Chemical              2              72   
3  ['Dubutamine', 'dobutamine']     Chemical              2              72   
4  ['Dubutamine', 'dobutamine']     Chemical              2              72   

  entity1_length entity2_id                        

In [25]:
unique_relations = merged_df_test['relation_type'].unique()
print("Unique relation types:", unique_relations)
print("Number of unique relation types:", len(unique_relations))
relation_counts = merged_df_test['relation_type'].value_counts()
relation_counts2 = merged_df_train['relation_type'].value_counts()
print("Relation type counts test:")
print(relation_counts)
print("Relation type counts train:")
print(relation_counts2)
num_duplicates = merged_df_test.duplicated().sum()
print("Number of duplicate rows in the DataFrame:", num_duplicates)
print(len(merged_df_test))
print(len(merged_df_train))

Unique relation types: ['CID' 'No_Relation']
Number of unique relation types: 2
Relation type counts test:
relation_type
No_Relation    4454
CID            1064
Name: count, dtype: int64
Relation type counts train:
relation_type
No_Relation    4548
CID            1037
Name: count, dtype: int64
Number of duplicate rows in the DataFrame: 0
5518
5585


In [26]:
def generate_multiple_prompt_examples_single(df, n_prompts=3, seed=42):
    import random
    import numpy as np

    relation_types = df['relation_type'].unique()
    prompts = []

    for prompt_num in range(n_prompts):
        used_indices = set()
        examples = []
        example_num = 1
        random.seed(seed + prompt_num)
        np.random.seed(seed + prompt_num)

        for rel in relation_types:
            rel_rows = df[df['relation_type'] == rel]
            if rel_rows.empty:
                continue
            available_indices = list(set(rel_rows.index) - used_indices)
            if available_indices:
                idx = random.choice(available_indices)
            else:
                idx = random.choice(rel_rows.index)
            used_indices.add(idx)
            row = df.loc[idx]
            entities = f"Entity1: {row['entity1_text']} ({row['entity1_type']}), Entity2: {row['entity2_text']} ({row['entity2_type']})"
            example = (
                f"Example {example_num}\n"
                f"## Input:\n"
                f"Text: {row['passage_text']}\n"
                f"{entities}\n"
                f"## Output:\n"
                f"{entities}, Relation: {row['relation_type']}\n"
            )
            examples.append(example)
            example_num += 1
            if len(examples) >= 9:
                break

        prompt_text = "\n".join(examples)
        prompts.append(prompt_text)

    return prompts



In [27]:
def generate_multiple_prompt_examples_multi(df, n_prompts=1, seed=42, number_relations=True, relations_entitys_separately=False):
    import random
    import numpy as np

    relation_types = df['relation_type'].unique()
    prompts = []

    for prompt_num in range(n_prompts):
        used_docids = set()
        examples = []
        example_num = 1
        random.seed(seed + prompt_num)
        np.random.seed(seed + prompt_num)

        for rel in relation_types:
            rel_rows = df[df['relation_type'] == rel]
            if rel_rows.empty:
                continue
            available_indices = list(set(rel_rows.index) - used_docids)
            if available_indices:
                idx = random.choice(available_indices)
            else:
                idx = random.choice(rel_rows.index)
            doc_id = df.loc[idx, 'doc_id']
            if doc_id in used_docids:
                continue
            used_docids.add(doc_id)
            doc_rows = df[df['doc_id'] == doc_id]

            passage_text = doc_rows.iloc[0]['passage_text']
            # List all entity pairs
            if number_relations:
                entity_lines = [
                    f"{i+1}. Entity1: {row['entity1_text']} ({row['entity1_type']}), Entity2: {row['entity2_text']} ({row['entity2_type']})"
                    for i, (_, row) in enumerate(doc_rows.iterrows())
                ]
            else:
                entity_lines = [
                    f"Entity1: {row['entity1_text']} ({row['entity1_type']}), Entity2: {row['entity2_text']} ({row['entity2_type']})"
                    for _, row in doc_rows.iterrows()
                ]
            if number_relations:
                if relations_entitys_separately:
                    relation_lines = [
                        f"{i+1}. Relation: {row['relation_type']}"
                        for i, (_, row) in enumerate(doc_rows.iterrows())
                    ]
                else:
                    relation_lines = [
                        f"{i+1}. Entity1: {row['entity1_text']}, Entity2: {row['entity2_text']}, Relation: {row['relation_type']}"
                        for i, (_, row) in enumerate(doc_rows.iterrows())
                    ]
            else:
                relation_lines = [
                    f"Entity1: {row['entity1_text']}, Entity2: {row['entity2_text']}, Relation: {row['relation_type']}"
                    for _, row in doc_rows.iterrows()
                ]
            example = (
                f"Example {example_num}\n"
                #f"DocID: {doc_id}\n"
                f"## Input:\n"
                f"Text: {passage_text}\n"
                f"Entities:\n" + "\n".join(entity_lines) + "\n"
                f"## Output:\n" + "\n".join(relation_lines) + "\n"
            )
            examples.append(example)
            example_num += 1
            if len(examples) >= 9:
                break

        prompt_text = "\n".join(examples)
        prompts.append(prompt_text)

    return prompts


In [28]:
def fill_prompt(template_path, text, e1="", entity_type1="", e2="", entity_type2="", examples="", entity_list="", prompt_id=None):
    with open(template_path, "r", encoding="utf-8") as f:
        template = f.read()
    filled = template.format(
        text=text,
        e1=e1,
        entity_type1=entity_type1,
        e2=e2,
        entity_type2=entity_type2,
        examples=examples,
        entity_list=entity_list,
        prompt_id=prompt_id
    )
    return filled


def create_prompt_relation_df_single(df,df_2, template_path, examples=False,same_examples=False, seed=42, n_prompts=1, prompt_id=False):
    import pandas as pd
    prompts = []
    relations = []
    doc_ids = []
    runs = []
    prompt_ids = []

    prompt_id_counter = 1  # Start numbering from 1

    for run_idx in range(1, n_prompts + 1):
        if examples:
            if same_examples == True:
                example_prompts = generate_multiple_prompt_examples_single(df_2, n_prompts=1, seed=seed)
                examples_str = example_prompts[0] if example_prompts else ""
            else:
            # Generate new examples for this run with updated seed
                example_prompts = generate_multiple_prompt_examples_single(df_2, n_prompts=1, seed=seed + run_idx - 1)
            # If you want to use these as context/examples, you can pass them to fill_prompt as needed
            # Here, we assume you want to add them as an 'examples' variable in the template
                examples_str = example_prompts[0] if example_prompts else ""
        else:
            examples_str = ""

        for _, row in df.iterrows():
            if prompt_id:
                prompt = fill_prompt(
                    template_path,
                    text=row['passage_text'],
                    e1=row['entity1_text'],
                    entity_type1=row['entity1_type'],
                    e2=row['entity2_text'],
                    entity_type2=row['entity2_type'],
                    examples=examples_str,
                    prompt_id=prompt_id_counter
                )
                prompts.append(prompt)
                relations.append(row['relation_type'])
                doc_ids.append(row['doc_id'])
                runs.append(run_idx)
                prompt_ids.append(prompt_id_counter)
                prompt_id_counter += 1
            else:
                prompt = fill_prompt(
                    template_path,
                    text=row['passage_text'],
                    e1=row['entity1_text'],
                    entity_type1=row['entity1_type'],
                    e2=row['entity2_text'],
                    entity_type2=row['entity2_type'],
                    examples=examples_str,
                    prompt_id=""
                )
                prompts.append(prompt)
                relations.append(row['relation_type'])
                doc_ids.append(row['doc_id'])
                runs.append(run_idx)
                prompt_ids.append(prompt_id_counter)
                prompt_id_counter += 1

    result_df = pd.DataFrame({
        'Prompt': prompts,
        'Relation': relations,
        'DocID': doc_ids,
        'run': runs,
        'prompt_id': prompt_ids
    })
    return result_df




def prompts_df_to_json_from_runs(df, n_key="n_1"):
    """
    Converts a DataFrame with 'Prompt', 'Relation', 'DocID', and 'run' columns to the required JSON structure.
    Each run contains lists for prompts, relations, and doc_ids.
    """
    runs = {}
    for run_num in sorted(df['run'].unique()):
        run_df = df[df['run'] == run_num]
        prompts_list = run_df['Prompt'].tolist()
        relations_list = run_df['Relation'].tolist()
        doc_ids_list = run_df['DocID'].tolist()
        runs[f"run_{run_num}"] = {
            "prompts1": prompts_list,
            "relations": relations_list,
            "doc_ids": doc_ids_list
        }

    result = {n_key: runs}
    return result

def prompts_df_to_json_from_runs_multi(df, n_key="n_1"):
    """
    Converts a DataFrame with 'Prompt', 'Relation'/'Multi_Relations', 'DocID'/'Multi_DocID', and 'run' columns
    to the required JSON structure. Each run contains lists for prompts, relations, and doc_ids.
    The JSON keys will match the DataFrame column names (e.g., 'Multi_Relations' -> 'multi_relations').
    """
    runs = {}
    # Find the correct column names for relations and doc_ids
    rel_col = None
    docid_col = None
    for c in df.columns:
        if c.lower().startswith("relation"):
            rel_col = c
        if c.lower().startswith("multi_relation"):
            rel_col = c
        if c.lower() == "relations":
            rel_col = c
        if c.lower() == "multi_relations":
            rel_col = c
        if c.lower().startswith("docid"):
            docid_col = c
        if c.lower().startswith("multi_docid"):
            docid_col = c
        if c.lower() == "doc_ids":
            docid_col = c
        if c.lower() == "multi_docid":
            docid_col = c

    # Fallbacks if not found
    if rel_col is None:
        rel_col = "Relation"
    if docid_col is None:
        docid_col = "DocID"

    # Use lower-case JSON keys for consistency
    rel_json_key = rel_col.lower()
    docid_json_key = docid_col.lower()

    for run_num in sorted(df['run'].unique()):
        run_df = df[df['run'] == run_num]
        prompts_list = run_df['Prompt'].tolist()
        relations_list = run_df[rel_col].tolist()
        doc_ids_list = run_df[docid_col].tolist()
        runs[f"run_{run_num}"] = {
            "prompts1": prompts_list,
            rel_json_key: relations_list,
            docid_json_key: doc_ids_list
        }

    result = {n_key: runs}
    return result



import json
#prompt_json = prompts_df_to_json_from_runs(prompt_relation_df)
#with open("prompts_for_api.json", "w") as f:
#    json.dump(prompt_json, f, indent=2)

In [39]:


import random
import pandas as pd




def create_prompt_relation_df_multi_by_docid(
    df, df_2, template_path, examples=False, same_examples=False,
    number_relations=False, relations_entitys_separately=False,
    seed=42, n_prompts=1, shuffle=False, new_shuffle_each_run = False
):
    """
    Create prompts for multi-relation extraction grouped by doc_id.
    
    Args:
        df: DataFrame with entity pairs to create prompts for
        df_2: DataFrame to use for generating few-shot examples
        template_path: Path to the prompt template file
        examples: Whether to include few-shot examples
        same_examples: Whether to use the same examples across all runs
        number_relations: Whether to number the entity pairs in the prompt
        relations_entitys_separately: Whether to show relations separately from entities
        seed: Random seed for reproducibility
        n_prompts: Number of runs to generate
        shuffle: If True, shuffle entity pairs within each doc_id for each run.
                 Different runs will have different shuffle orders for the same doc_id.
    
    Returns:
        DataFrame with columns: Prompt, Multi_Relations, Multi_DocID, run
    """
    import pandas as pd
    import random

    prompts = []
    relations = []
    doc_ids = []
    runs = []
    shuffle_seed = seed
    # Group by doc_id
    grouped = df.groupby('doc_id')

    for run_idx in range(1, n_prompts + 1):
        if examples:
            if same_examples:
                example_prompts = generate_multiple_prompt_examples_multi(
                    df_2, n_prompts=1, seed=seed,
                    number_relations=number_relations,
                    relations_entitys_separately=relations_entitys_separately
                )
                examples_str = example_prompts[0] if example_prompts else ""
            else:
                example_prompts = generate_multiple_prompt_examples_multi(
                    df_2, n_prompts=1, seed=seed + run_idx - 1,
                    number_relations=number_relations,
                    relations_entitys_separately=relations_entitys_separately
                )
                examples_str = example_prompts[0] if example_prompts else ""
        else:
            examples_str = ""

        for doc_id, doc_rows in grouped:
            # If shuffle is enabled, shuffle the rows for this doc_id
            # Use a seed based on: base_seed + run_idx + hash(doc_id) for reproducibility
            # and different shuffle per run for the same doc_id
            if shuffle:
                if new_shuffle_each_run == True: ## check if we want to shuffle for each run 
                    shuffle_seed = seed + run_idx + hash(str(doc_id)) % 10000
                else:
                    shuffle_seed = seed

                random.seed(shuffle_seed)
                # Shuffle the indices and reorder doc_rows
                shuffled_indices = list(range(len(doc_rows)))
                random.shuffle(shuffled_indices)
                doc_rows = doc_rows.iloc[shuffled_indices].reset_index(drop=True)

            # Build entity_lines for this doc_id
            if number_relations:
                entity_lines = [
                    f"{i+1}. Entity1: {row['entity1_text']} ({row['entity1_type']}), Entity2: {row['entity2_text']} ({row['entity2_type']})"
                    for i, (_, row) in enumerate(doc_rows.iterrows())
                ]
            else:
                entity_lines = [
                    f"Entity1: {row['entity1_text']} ({row['entity1_type']}), Entity2: {row['entity2_text']} ({row['entity2_type']})"
                    for _, row in doc_rows.iterrows()
                ]
            # Collect relations for this doc_id (in the same order as entity_lines)
            relation_list = doc_rows['relation_type'].tolist()

            # Use the first passage_text for the prompt
            passage_text = doc_rows.iloc[0]['passage_text']

            prompt = fill_prompt(
                template_path,
                text=passage_text,
                entity_list="\n".join(entity_lines),
                examples=examples_str
            )
            prompts.append(prompt)
            relations.append(relation_list)
            doc_ids.append(doc_id)
            runs.append(run_idx)

    result_df = pd.DataFrame({
        'Prompt': prompts,
        'Multi_Relations': relations,
        'Multi_DocID': doc_ids,
        'run': runs
    })
    return result_df

# Alternative function or shuffle for even split of realtions

In [20]:



import pandas as pd
import numpy as np

def print_summary_df(df):

        # Step 1: Group by 'doc_id' and get the size of each group
        group_sizes = df.groupby('doc_id').size()
        size_counts = group_sizes.value_counts().sort_index()
        total_groups = len(group_sizes)
        size_percentages = (size_counts / total_groups) * 100

        # Create a DataFrame for group sizes
        group_sizes_df = pd.DataFrame({
        'Number of Rows': size_counts.index,
        'Count': size_counts.values,
        'Percentage': size_percentages.values
        }).set_index('Number of Rows')

        # Step 2: Calculate label distribution for each group length
        doc_rows = df.merge(
        group_sizes.reset_index(name='group_length'),
        on='doc_id'
        )
        label_distribution = (
        doc_rows
        .groupby(['group_length', 'relation_type'])
        .size()
        .unstack(fill_value=0)
        )
        label_percentages = label_distribution.div(label_distribution.sum(axis=1), axis=0) * 100

        # Step 3: Merge both results
        result = pd.concat([group_sizes_df, label_percentages], axis=1)

        print(result)

def balance_group(group):
    cid = group[group['relation_type'] == 'CID']
    no_rel = group[group['relation_type'] == 'No_Relation']

    # Calculate the desired number of rows for each label
    min_len = min(len(cid), len(no_rel) // 2)  # Ensure at least 1:2 ratio
    desired_cid = min_len
    desired_no_rel = 2 * min_len

    # Randomly sample the desired number of rows
    balanced_cid = cid.sample(n=desired_cid, random_state=42, replace=False)
    balanced_no_rel = no_rel.sample(n=desired_no_rel, random_state=42, replace=False)

    # Combine and return
    return pd.concat([balanced_cid, balanced_no_rel])
"""
def balance_group(group):
    cid = group[group['relation_type'] == 'CID']
    no_rel = group[group['relation_type'] == 'No_Relation']

    # Calculate the desired number of rows for each label (50/50)
    min_len = min(len(cid), len(no_rel))  # Ensure equal number of rows

    # Randomly sample the desired number of rows
    balanced_cid = cid.sample(n=min_len, random_state=42, replace=False)
    balanced_no_rel = no_rel.sample(n=min_len, random_state=42, replace=False)

    # Combine and return
    return pd.concat([balanced_cid, balanced_no_rel])
"""
import pandas as pd
def show_index_table(df):
# Step 1: Assign an index to each row within its group
        df['row_index'] = df.groupby('doc_id').cumcount()

        # Step 2: For each index position, count the labels
        label_distribution = (
        df
        .groupby(['row_index', 'relation_type'])
        .size()
        .unstack(fill_value=0)
        )
        # Step 3: Calculate the percentage distribution for each index position
        label_percentages = label_distribution.div(label_distribution.sum(axis=1), axis=0) * 100

        print(label_percentages)

import pandas as pd

def filter_and_truncate_groups(df, max, min):
    # Step 1: Calculate the size of each group
    group_sizes = df.groupby('doc_id').size()

    # Step 2: Filter out groups with fewer than x rows
    #valid_groups = group_sizes[group_sizes >= min].index
    #filtered_df = df[df['doc_id'].isin(valid_groups)]

    # Step 3: Truncate groups with more than x rows to exactly x rows
    truncated_df = df.groupby('doc_id').head(max)

    return truncated_df

import pandas as pd

def balance_group_with_margin(group, target_pct_cid=0.33, target_pct_no_rel=0.66, margin=0):
    cid = group[group['relation_type'] == 'CID']
    no_rel = group[group['relation_type'] == 'No_Relation']

    len_cid = len(cid)
    len_no_rel = len(no_rel)
    total = len_cid + len_no_rel

    # Calculate current percentages
    current_pct_cid = len_cid / total
    current_pct_no_rel = len_no_rel / total

    # Check if current proportions are within the margin
    if (abs(current_pct_cid - target_pct_cid) <= margin and
        abs(current_pct_no_rel - target_pct_no_rel) <= margin):
        return group  # Keep all rows if within margin

    # If not, sample to get as close as possible to the target
    desired_cid = round(total * target_pct_cid)
    desired_no_rel = total - desired_cid

    # Ensure we don't sample more than available
    desired_cid = min(desired_cid, len_cid)
    desired_no_rel = min(desired_no_rel, len_no_rel)

    # Sample
    balanced_cid = cid.sample(n=desired_cid, random_state=42, replace=False)
    balanced_no_rel = no_rel.sample(n=desired_no_rel, random_state=42, replace=False)

    return pd.concat([balanced_cid, balanced_no_rel])


# Assuming df1 is the DataFrame with fewer columns,
# and df2 is the DataFrame with 3 extra columns.
columns_to_keep = merged_df_train.columns  # Columns present in df1
merged_df_dev = merged_df_dev[columns_to_keep]     # Drop extra columns in df2
df = pd.concat([merged_df_train, merged_df_dev], ignore_index=True)

max_c = 18  # Your target number of rows per group
min_c = 6
#df = merged_df_train
grouped = df.groupby('doc_id')
df_cut = filter_and_truncate_groups(df, max=max_c, min=min_c)

# balance data
## balance data after cut
balanced_df_cut = df_cut.groupby('doc_id').apply(balance_group).reset_index(drop=True)
balanced_df_cut = balanced_df_cut.groupby('doc_id').filter(lambda x: len(x) > 0)
#balance data before cut
balanced_df = df.groupby('doc_id').apply(balance_group).reset_index(drop=True)
#balanced_df = balanced_df.groupby('doc_id').filter(lambda x: len(x) > 0)
balanced_df = filter_and_truncate_groups(balanced_df, max=max_c, min=min_c)

#print_summary_df(balanced_df)

# Step 1: Shuffle rows within each group
shuffled_df = balanced_df_cut.groupby('doc_id').apply(lambda x: x.sample(frac=1, random_state=np.random.randint(1, 100))).reset_index(drop=True)

# Step 2: Reassign row_index after shuffling
shuffled_df['row_index'] = shuffled_df.groupby('doc_id').cumcount()


show_index_table(shuffled_df)
print_summary_df(shuffled_df)
#show_index_table(df)


#for doc_id, doc_rows in grouped:
        #print(doc_id)
        #print(doc_rows["relation_type"])



/tmp/ipykernel_362665/777292713.py:147: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df_cut = df_cut.groupby('doc_id').apply(balance_group).reset_index(drop=True)
/tmp/ipykernel_362665/777292713.py:150: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df = df.groupby('doc_id').apply(balance_group).reset_index(drop=True)


relation_type        CID  No_Relation
row_index                            
0              37.076023    62.923977
1              31.812865    68.187135
2              29.356725    70.643275
3              30.681818    69.318182
4              32.386364    67.613636
5              33.806818    66.193182
6              34.814815    65.185185
7              44.444444    55.555556
8              37.037037    62.962963
9              31.578947    68.421053
10             28.947368    71.052632
11             35.526316    64.473684
12             47.058824    52.941176
13             23.529412    76.470588
14             44.117647    55.882353
15             50.000000    50.000000
16             42.857143    57.142857
17             21.428571    78.571429
    Count  Percentage        CID  No_Relation
3     503   58.830409  33.333333    66.666667
6     217   25.380117  33.333333    66.666667
9      59    6.900585  33.333333    66.666667
12     42    4.912281  33.333333    66.666667
15     20 

/tmp/ipykernel_362665/777292713.py:157: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  shuffled_df = balanced_df_cut.groupby('doc_id').apply(lambda x: x.sample(frac=1, random_state=np.random.randint(1, 100))).reset_index(drop=True)


In [37]:
import pandas as pd
import numpy as np
from collections import defaultdict

def optimize_index_positions(df, target_pct_cid=0.33, target_pct_no_rel=0.66, max_iterations=100):
    """
    Strategically rearrange rows within each group to minimize deviation from
    target label distribution at each index position across all groups.
    
    Parameters:
    -----------
    df : DataFrame
        Input dataframe with 'doc_id' and 'relation_type' columns
    target_pct_cid : float
        Target percentage for CID labels (default 0.33)
    target_pct_no_rel : float
        Target percentage for No_Relation labels (default 0.66)
    max_iterations : int
        Maximum optimization iterations (default 100)
    
    Returns:
    --------
    DataFrame with optimized row positions
    """
    
    # Create a copy to avoid modifying original
    result_df = df.copy()
    
    # Get all unique groups
    groups = result_df.groupby('doc_id')
    
    # Store rows by group with their labels
    group_data = {}
    for doc_id, group in groups:
        group_data[doc_id] = group.copy()
    
    # Find maximum group size to know how many index positions we have
    max_group_size = max(len(g) for g in group_data.values())
    
    # Initialize: assign initial positions (can be random or sequential)
    for doc_id in group_data:
        group_data[doc_id] = group_data[doc_id].reset_index(drop=True)
    
    # Optimization loop
    for iteration in range(max_iterations):
        improved = False
        
        # For each index position
        for idx_pos in range(max_group_size):
            # Collect all groups that have this index position
            groups_with_idx = {doc_id: g for doc_id, g in group_data.items() 
                              if len(g) > idx_pos}
            
            if len(groups_with_idx) < 2:
                continue
            
            # Calculate current distribution at this index
            current_labels = [g.iloc[idx_pos]['relation_type'] 
                            for g in groups_with_idx.values()]
            current_cid_count = sum(1 for l in current_labels if l == 'CID')
            current_total = len(current_labels)
            current_pct = current_cid_count / current_total if current_total > 0 else 0
            
            # Calculate current deviation from target
            current_deviation = abs(current_pct - target_pct_cid)
            
            # Try swapping within groups to improve this position
            group_ids = list(groups_with_idx.keys())
            
            for i, doc_id in enumerate(group_ids):
                group = group_data[doc_id]
                current_label_at_idx = group.iloc[idx_pos]['relation_type']
                
                # Try swapping with other positions in the same group
                for swap_pos in range(len(group)):
                    if swap_pos == idx_pos:
                        continue
                    
                    swap_label = group.iloc[swap_pos]['relation_type']
                    
                    # Skip if labels are the same (no point swapping)
                    if current_label_at_idx == swap_label:
                        continue
                    
                    # Calculate what the new distribution would be after swap
                    new_labels = current_labels.copy()
                    new_labels[i] = swap_label
                    new_cid_count = sum(1 for l in new_labels if l == 'CID')
                    new_pct = new_cid_count / current_total
                    new_deviation = abs(new_pct - target_pct_cid)
                    
                    # If this swap improves the distribution, do it
                    if new_deviation < current_deviation:
                        # Perform the swap
                        group_data[doc_id].iloc[idx_pos], group_data[doc_id].iloc[swap_pos] = \
                            group_data[doc_id].iloc[swap_pos].copy(), group_data[doc_id].iloc[idx_pos].copy()
                        
                        current_deviation = new_deviation
                        current_labels = new_labels
                        improved = True
        
        # If no improvement in this iteration, we've reached a local optimum
        if not improved:
            break
    
    # Combine all groups back together
    result_frames = []
    for doc_id, group in group_data.items():
        group = group.copy()
        group['row_index'] = range(len(group))
        result_frames.append(group)
    
    result_df = pd.concat(result_frames, ignore_index=True)
    
    return result_df


def greedy_index_balance(df, target_pct_cid=0.33):
    """
    Alternative greedy approach: build the arrangement from scratch,
    placing each row in the position that best balances the distribution.
    
    This is faster but may not find the absolute best solution.
    """
    
    # Group by doc_id
    groups = {}
    for doc_id, group in df.groupby('doc_id'):
        groups[doc_id] = group.copy().reset_index(drop=True)
    
    # Find max group size
    max_size = max(len(g) for g in groups.values())
    
    # For each group, create a list to hold the final order
    final_orders = {doc_id: [None] * len(group) for doc_id, group in groups.items()}
    
    # Track which rows have been placed
    placed = {doc_id: [False] * len(group) for doc_id, group in groups.items()}
    
    # For each index position, greedily assign rows
    for idx_pos in range(max_size):
        # Track current distribution at this position
        position_labels = []
        
        # Get groups that have this position available
        for doc_id, group in groups.items():
            if len(group) <= idx_pos:
                continue
            
            # Find best unplaced row to put at this position
            best_row_idx = None
            best_deviation = float('inf')
            
            for row_idx in range(len(group)):
                if placed[doc_id][row_idx]:
                    continue
                
                # Try this row at this position
                test_label = group.iloc[row_idx]['relation_type']
                test_labels = position_labels + [test_label]
                
                # Calculate deviation
                cid_count = sum(1 for l in test_labels if l == 'CID')
                total = len(test_labels)
                pct = cid_count / total if total > 0 else 0
                deviation = abs(pct - target_pct_cid)
                
                if deviation < best_deviation:
                    best_deviation = deviation
                    best_row_idx = row_idx
            
            # Place the best row at this position
            if best_row_idx is not None:
                final_orders[doc_id][idx_pos] = group.iloc[best_row_idx]
                placed[doc_id][best_row_idx] = True
                position_labels.append(group.iloc[best_row_idx]['relation_type'])
    
    # Reconstruct dataframe
    result_frames = []
    for doc_id, ordered_rows in final_orders.items():
        # Filter out None values (for groups smaller than max_size)
        ordered_rows = [r for r in ordered_rows if r is not None]
        if ordered_rows:
            group_df = pd.DataFrame(ordered_rows)
            group_df['row_index'] = range(len(group_df))
            result_frames.append(group_df)
    
    return pd.concat(result_frames, ignore_index=True)


# Example usage to replace your random shuffle:
# Instead of:
# shuffled_df = balanced_df_cut.groupby('doc_id').apply(lambda x: x.sample(frac=1, random_state=np.random.randint(1, 100))).reset_index(drop=True)

# Use:
#optimized_df = optimize_index_positions(balanced_df_cut, target_pct_cid=0.33, target_pct_no_rel=0.66)

# Or for faster execution:
optimized_df = greedy_index_balance(balanced_df_cut, target_pct_cid=0.33)
show_index_table(optimized_df)
print_summary_df(optimized_df)
optimized_df.head(20)

relation_type        CID  No_Relation
row_index                            
0              32.982456    67.017544
1              32.982456    67.017544
2              33.099415    66.900585
3              32.954545    67.045455
4              32.954545    67.045455
5              33.238636    66.761364
6              33.333333    66.666667
7              33.333333    66.666667
8              35.555556    64.444444
9              34.210526    65.789474
10             35.526316    64.473684
11             36.842105    63.157895
12             35.294118    64.705882
13             35.294118    64.705882
14             35.294118    64.705882
15             28.571429    71.428571
16             28.571429    71.428571
17             50.000000    50.000000
    Count  Percentage        CID  No_Relation
3     503   58.830409  33.333333    66.666667
6     217   25.380117  33.333333    66.666667
9      59    6.900585  33.333333    66.666667
12     42    4.912281  33.333333    66.666667
15     20 

,doc_id,passage_text,entity1_id,entity1_text,entity1_type,entity1_ann_id,entity1_offset,entity1_length,entity2_id,entity2_text,entity2_type,entity2_ann_id,entity2_offset,entity2_length,relation_type,is_novel,row_index
0,10027919,22-oxacalcitriol suppresses secondary hyperpar...,D002117,['Calcitriol'],Chemical,4,133.0,10,D005355,['fibrosis'],Disease,28,1988.0,8,No_Relation,Unknown,0
1,10027919,22-oxacalcitriol suppresses secondary hyperpar...,D014807,['vitamin D'],Chemical,9,378.0,9,D001851,"['adynamic bone disease', 'low bone turnover',...",Disease,2,75.0,17,No_Relation,Unknown,1
2,10027919,22-oxacalcitriol suppresses secondary hyperpar...,C051883,"['22-oxacalcitriol', 'OCT']",Chemical,0,0.0,16,D006962,['secondary hyperparathyroidism'],Disease,1,28.0,29,No_Relation,Unknown,2
3,10027919,22-oxacalcitriol suppresses secondary hyperpar...,D014807,['vitamin D'],Chemical,9,378.0,9,D051437,"['renal failure', 'renal insufficiency']",Disease,3,106.0,13,No_Relation,Unknown,3
4,10027919,22-oxacalcitriol suppresses secondary hyperpar...,D002117,['Calcitriol'],Chemical,4,133.0,10,D006962,['secondary hyperparathyroidism'],Disease,1,28.0,29,No_Relation,Unknown,4
5,10027919,22-oxacalcitriol suppresses secondary hyperpar...,D002117,['Calcitriol'],Chemical,4,133.0,10,D054559,['hyperphosphatemia'],Disease,24,1721.0,17,No_Relation,Unknown,5
6,10027919,22-oxacalcitriol suppresses secondary hyperpar...,C051883,"['22-oxacalcitriol', 'OCT']",Chemical,0,0.0,16,D051437,"['renal failure', 'renal insufficiency']",Disease,3,106.0,13,No_Relation,Unknown,6
7,10027919,22-oxacalcitriol suppresses secondary hyperpar...,C051883,"['22-oxacalcitriol', 'OCT']",Chemical,0,0.0,16,D001851,"['adynamic bone disease', 'low bone turnover',...",Disease,2,75.0,17,No_Relation,Unknown,7
8,10027919,22-oxacalcitriol suppresses secondary hyperpar...,C051883,"['22-oxacalcitriol', 'OCT']",Chemical,0,0.0,16,D054559,['hyperphosphatemia'],Disease,24,1721.0,17,CID,Unknown,8
9,10027919,22-oxacalcitriol suppresses secondary hyperpar...,D002117,['Calcitriol'],Chemical,4,133.0,10,D006934,['hypercalcemia'],Disease,6,273.0,13,CID,Unknown,9


In [54]:
def balance_labels_by_index(df):
    """
    Balance labels across groups by index position.
    
    For each index position across all groups:
    - Drop labels until achieving 33% CID and 66% No_Relation ratio
    - Drop from groups with most entries first
    - If ratio can't be achieved, drop the entire index position
    - Re-index groups after balancing
    
    Args:
        df: DataFrame with columns 'doc_id', 'relation_type'
        target_cid_ratio: Target ratio for CID labels (default 0.33)
        target_no_relation_ratio: Target ratio for No_Relation labels (default 0.66)
    
    Returns:
        Balanced DataFrame with re-indexed groups
    """
    # Add index within each doc_id group
    df = df.copy()
    df['group_index'] = df.groupby('doc_id').cumcount()
    
    # Get group sizes for sorting (drop from largest groups first)
    group_sizes = df.groupby('doc_id').size().to_dict()
    df['group_size'] = df['doc_id'].map(group_sizes)
    
    rows_to_keep = []
    
    # Process each index position
    max_index = df['group_index'].max()
    
    for idx in range(max_index + 1):
        idx_data = df[df['group_index'] == idx].copy()
        
        if len(idx_data) == 0:
            continue
        
        # Count labels at this index
        label_counts = idx_data['relation_type'].value_counts()
        
        # Check if we have both CID and No_Relation
        if 'CID' not in label_counts or 'No_Relation' not in label_counts:
            # Skip this index if we don't have both required labels
            continue
        
        cid_count = label_counts.get('CID', 0)
        no_relation_count = label_counts.get('No_Relation', 0)
        other_count = len(idx_data) - cid_count - no_relation_count
        
        # Calculate target counts based on CID count
        # If we have X CID labels, we need 2X No_Relation labels (to get 33%/66% ratio)

        target_no_relation = int(cid_count * 2)
   
        target_cid_relation = int(no_relation_count / 2)
        
        
        # Sort by group_size (descending) so we drop from largest groups first
        idx_data = idx_data.sort_values('group_size', ascending=False)
        
        # filter CID
        cid_rows = idx_data[idx_data['relation_type'] == 'CID'].head(target_cid_relation) # this does nothing why ?
        
        # Keep only target_no_relation No_Relation labels
        no_relation_rows = idx_data[idx_data['relation_type'] == 'No_Relation'].head(target_no_relation)
        
        # Drop all other labels
        balanced_idx_data = pd.concat([cid_rows, no_relation_rows])
        
        rows_to_keep.append(balanced_idx_data)
    
    # Combine all kept rows
    if not rows_to_keep:
        return pd.DataFrame(columns=df.columns)
    
    balanced_df = pd.concat(rows_to_keep, ignore_index=True)
    
    # Remove helper columns
    balanced_df = balanced_df.drop(columns=['group_index', 'group_size'])
    
    # Re-index within each doc_id group
    balanced_df['new_index'] = balanced_df.groupby('doc_id').cumcount()
    
    # Optional: sort by doc_id and new_index for clarity
    balanced_df = balanced_df.sort_values(['doc_id', 'new_index']).reset_index(drop=True)
    
    # Drop the new_index column if you don't need it in the output
    balanced_df = balanced_df.drop(columns=['new_index'])
    
    return balanced_df


# Usage example:
balanced_df = balance_labels_by_index(df)

# Check the results
#print("Original data:")
#print_summary_df(df)
print("\nBalanced data:")
#print_summary_df(balanced_df)
show_index_table(balanced_df)

def filter_by_no_relation_percentage(df, min_pct=61, max_pct=71):
    """
    Drop entire index positions across all doc_ids if the No_Relation 
    percentage is less than min_pct or more than max_pct.
    
    Args:
        df: DataFrame with columns 'doc_id', 'relation_type'
        min_pct: Minimum percentage of No_Relation labels (default 61)
        max_pct: Maximum percentage of No_Relation labels (default 71)
    
    Returns:
        Filtered DataFrame
    """
    # Add index within each doc_id group
    df = df.copy()
    df['group_index'] = df.groupby('doc_id').cumcount()
    
    # Calculate No_Relation percentage for each index position
    index_stats = df.groupby('group_index')['relation_type'].apply(
        lambda x: (x == 'No_Relation').sum() / len(x) * 100
    ).reset_index()
    index_stats.columns = ['group_index', 'no_relation_pct']
    
    # Find valid index positions
    valid_indices = index_stats[
        (index_stats['no_relation_pct'] >= min_pct) & 
        (index_stats['no_relation_pct'] <= max_pct)
    ]['group_index'].values
    
    # Filter dataframe to keep only valid index positions
    filtered_df = df[df['group_index'].isin(valid_indices)].copy()
    
    # Remove helper column
    filtered_df = filtered_df.drop(columns=['group_index'])
    
    # Re-index within each doc_id group
    filtered_df['new_index'] = filtered_df.groupby('doc_id').cumcount()
    filtered_df = filtered_df.sort_values(['doc_id', 'new_index']).reset_index(drop=True)
    filtered_df = filtered_df.drop(columns=['new_index'])
    
    return filtered_df

# Usage:
balanced_df = balance_labels_by_index(df)
final_df = filter_by_no_relation_percentage(balanced_df, min_pct=61, max_pct=71)

print("\nFinal filtered data:")
print_summary_df(final_df)
show_index_table(final_df)


Balanced data:
relation_type        CID  No_Relation
row_index                            
0              37.954545    62.045455
1              34.482759    65.517241
2              29.691877    70.308123
3              33.762058    66.237942
4              31.617647    68.382353
5              33.018868    66.981132
6              34.911243    65.088757
7              42.148760    57.851240
8              38.750000    61.250000
9              35.820896    64.179104
10             40.740741    59.259259
11             30.952381    69.047619
12             26.470588    73.529412
13             22.580645    77.419355
14             25.925926    74.074074
15             16.666667    83.333333
16             25.000000    75.000000
17             29.411765    70.588235
18             12.500000    87.500000
19             20.000000    80.000000
20             20.000000    80.000000
21             22.222222    77.777778
22             25.000000    75.000000
23             16.666667    83.333

In [38]:
template_path_multi = "LLM_benchmarks/prompt_template_multi.txt"
template_path_single = "LLM_benchmarks/prompt_template_single.txt"
save_as = "LLM_benchmarks/Prompts/few_shot_single_for_api.json"
merged_df_train = optimized_df
zero_shot_single = create_prompt_relation_df_single(merged_df_train,merged_df_test, template_path_single, examples=False,same_examples=True, seed=42, n_prompts=3)
print(zero_shot_single.head())
import json
prompt_json = prompts_df_to_json_from_runs(zero_shot_single)
with open("LLM_benchmarks/Prompts/zero_shot_single_for_api_2_CID.json", "w") as f:
    json.dump(prompt_json, f, indent=2)

few_shot_single = create_prompt_relation_df_single(merged_df_train,merged_df_test, template_path_single, examples=True,same_examples=True, seed=42, n_prompts=3)
print(few_shot_single.head())
prompt_json = prompts_df_to_json_from_runs(few_shot_single)
with open("LLM_benchmarks/Prompts/few_shot_single_for_api_2_CID.json", "w") as f:
    json.dump(prompt_json, f, indent=2)


# Run garbage collection to free up memory
import gc
gc.collect()

zero_shot_multi = create_prompt_relation_df_multi_by_docid(merged_df_train,merged_df_test, template_path_multi, examples=False,same_examples=True, seed=42, n_prompts=3, number_relations=True, relations_entitys_separately=False)
prompt_json = prompts_df_to_json_from_runs_multi(zero_shot_multi)
with open("LLM_benchmarks/Prompts/zero_shot_multi_for_api_2_CID.json", "w") as f:
    json.dump(prompt_json, f, indent=2)

few_shot_multi = create_prompt_relation_df_multi_by_docid(merged_df_train,merged_df_test, template_path_multi, examples=True,same_examples=True, seed=42, n_prompts=3, number_relations=True, relations_entitys_separately=False)
prompt_json = prompts_df_to_json_from_runs_multi(few_shot_multi)
with open("LLM_benchmarks/Prompts/few_shot_multi_for_api_2_CID.json", "w") as f:
    json.dump(prompt_json, f, indent=2) 
print(few_shot_multi.head())




                                              Prompt     Relation     DocID  \
0  You are a biomedical relation extraction exper...  No_Relation  10027919   
1  You are a biomedical relation extraction exper...  No_Relation  10027919   
2  You are a biomedical relation extraction exper...  No_Relation  10027919   
3  You are a biomedical relation extraction exper...  No_Relation  10027919   
4  You are a biomedical relation extraction exper...  No_Relation  10027919   

   run  prompt_id  
0    1          1  
1    1          2  
2    1          3  
3    1          4  
4    1          5  
                                              Prompt     Relation     DocID  \
0  You are a biomedical relation extraction exper...  No_Relation  10027919   
1  You are a biomedical relation extraction exper...  No_Relation  10027919   
2  You are a biomedical relation extraction exper...  No_Relation  10027919   
3  You are a biomedical relation extraction exper...  No_Relation  10027919   
4  You are

In [40]:
# Generate SHUFFLED versions of multi prompts
# The shuffle=True parameter will shuffle entity pairs within each doc_id
# Different runs will have different shuffle orders for the same doc_id

# Zero-shot multi with shuffled entity order
zero_shot_multi_shuffled = create_prompt_relation_df_multi_by_docid(
    merged_df_train, merged_df_test, template_path_multi,
    examples=False, same_examples=False,
    seed=42, n_prompts=3,
    number_relations=True, relations_entitys_separately=False,
    shuffle=False
 # Enable shuffling
)
prompt_json = prompts_df_to_json_from_runs_multi(zero_shot_multi_shuffled)
with open("LLM_benchmarks/Prompts/zero_shot_multi_for_api_shuffled_2_even.json", "w") as f:
    json.dump(prompt_json, f, indent=2)
print("Saved: zero_shot_multi_for_api_shuffled.json")

# Few-shot multi with shuffled entity order
few_shot_multi_shuffled = create_prompt_relation_df_multi_by_docid(
    merged_df_train, merged_df_test, template_path_multi,
    examples=True, same_examples=True,
    seed=42, n_prompts=3,
    number_relations=True, relations_entitys_separately=False,
    shuffle=False
)
prompt_json = prompts_df_to_json_from_runs_multi(few_shot_multi_shuffled)
with open("LLM_benchmarks/Prompts/few_shot_multi_for_api_shuffled_2_even.json", "w") as f:
    json.dump(prompt_json, f, indent=2)
print("Saved: few_shot_multi_for_api_shuffled.json")


Saved: zero_shot_multi_for_api_shuffled.json
Saved: few_shot_multi_for_api_shuffled.json


## Make 1 big json for the prompts

In [41]:
def consolidate_prompt_jsons(json_paths, prompt_set_names, output_path):
    """
    Merge multiple prompt JSONs into one, each as a different prompt set name.
    Also adds Multi_Relations and Multi_DocID fields if present and not already in the base.
    """
    import json

    # Load all JSONs
    jsons = [json.load(open(p, "r", encoding="utf-8")) for p in json_paths]
    base = jsons[0]
    n_key = list(base.keys())[0]
    runs = base[n_key].keys()

    # For each run, add all prompt sets and Multi fields if present
    for run in runs:
        for i, js in enumerate(jsons):
            prompt_set_name = prompt_set_names[i]
            # Find the first prompt set key in this json (e.g. "prompts1")
            prompt_set_key = [k for k in js[n_key][run].keys() if k.startswith("prompts")][0]
            base[n_key][run][prompt_set_name] = js[n_key][run][prompt_set_key]
            # Add Multi_Relations if present and not already in base
            if "multi_relations" in js[n_key][run] and "multi_relations" not in base[n_key][run]:
                base[n_key][run]["multi_relations"] = js[n_key][run]["multi_relations"]
            # Add Multi_DocID if present and not already in base
            if "multi_docid" in js[n_key][run] and "multi_docid" not in base[n_key][run]:
                base[n_key][run]["multi_docid"] = js[n_key][run]["multi_docid"]

    # Save merged JSON
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(base, f, indent=2)



consolidate_prompt_jsons(
    [
        "LLM_benchmarks/Prompts/zero_shot_single_for_api_2_CID.json",
        "LLM_benchmarks/Prompts/few_shot_single_for_api_2_CID.json",
        "LLM_benchmarks/Prompts/zero_shot_multi_for_api_shuffled_2_even.json",
        "LLM_benchmarks/Prompts/few_shot_multi_for_api_shuffled_2_even.json"
    ],
    ["single_zero", "single_examples", "multi_zero", "multi_examples"],
    "LLM_benchmarks/Prompts/Test_callibration_CID_Shuffeled_3_even.json"
)


In [15]:
import numpy as np

# Set your seed for reproducibility
seed = 42
np.random.seed(seed)

# Get unique doc_ids
unique_docids = merged_df_train['doc_id'].unique()

# Pick 20 random doc_ids
sampled_docids = np.random.choice(unique_docids, size=20, replace=False)

# Subset the DataFrame to only rows with those doc_ids
subset_df = merged_df_train[merged_df_train['doc_id'].isin(sampled_docids)].copy()

print(subset_df.shape)
print(subset_df['doc_id'].unique())

template_path_multi = "LLM_benchmarks/prompt_template_multi_CID.txt"
template_path_single = "LLM_benchmarks/prompt_template_single_CID.txt"
#template_path_single_wo_reason = "LLM_benchmarks/prompt_template_single_wo_reason.txt"

test_single = create_prompt_relation_df_single(subset_df,merged_df_test, template_path_single, examples=False,same_examples=False, seed=42, n_prompts=25, prompt_id=False)
print(test_single.head())
import json
prompt_json = prompts_df_to_json_from_runs(test_single)
with open("LLM_benchmarks/Prompts/test_single_for_api_CID_2.json", "w") as f:
    json.dump(prompt_json, f, indent=2)

test_single_w_examples = create_prompt_relation_df_single(subset_df,merged_df_test, template_path_single, examples=True,same_examples=True, seed=42, n_prompts=25, prompt_id=False)
print(test_single_w_examples.head())
import json
prompt_json = prompts_df_to_json_from_runs(test_single_w_examples)
with open("LLM_benchmarks/Prompts/test_single_for_api_examples_CID_2.json", "w") as f:
    json.dump(prompt_json, f, indent=2)

test_multi = create_prompt_relation_df_multi_by_docid(subset_df,merged_df_test, template_path_multi, examples=False,same_examples=False, seed=42, n_prompts=25, number_relations=True, relations_entitys_separately=False)
print(test_multi.head())
import json
prompt_json = prompts_df_to_json_from_runs_multi(test_multi)
with open("LLM_benchmarks/Prompts/test_multi_for_api_CID_2.json", "w") as f:
    json.dump(prompt_json, f, indent=2) 

test_multi_w_examples = create_prompt_relation_df_multi_by_docid(subset_df,merged_df_test, template_path_multi, examples=True,same_examples=True, seed=42, n_prompts=25, number_relations=True, relations_entitys_separately=False)
print(test_multi_w_examples.head())
import json
prompt_json = prompts_df_to_json_from_runs_multi(test_multi_w_examples)
with open("LLM_benchmarks/Prompts/test_multi_for_api_examples_CID_2.json", "w") as f:
    json.dump(prompt_json, f, indent=2)

consolidate_prompt_jsons(
    [
        "LLM_benchmarks/Prompts/test_single_for_api_CID_2.json",
        "LLM_benchmarks/Prompts/test_single_for_api_examples_CID_2.json",
        "LLM_benchmarks/Prompts/test_multi_for_api_CID_2.json",
        "LLM_benchmarks/Prompts/test_multi_for_api_examples_CID_2.json"
    ],
    ["single_zero", "single_examples", "multi_zero", "multi_examples"],
    "LLM_benchmarks/Prompts/Test_callibration_CID_long.json"
)
#calibration_test = json.load(open("LLM_benchmarks/Prompts/Test_callibration_CID.json", "r", encoding="utf-8"))

(191, 16)
['10579464' '11250767' '12523465' '12596116' '12789195' '15036754'
 '15804801' '17042910' '18208574' '2870085' '3856631' '4812392' '6103707'
 '6209318' '6538499' '6747681' '7292072' '7420681' '8682684' '983936']
                                              Prompt     Relation     DocID  \
0  You are a biomedical relation extraction exper...  No_Relation  10579464   
1  You are a biomedical relation extraction exper...  No_Relation  10579464   
2  You are a biomedical relation extraction exper...  No_Relation  10579464   
3  You are a biomedical relation extraction exper...          CID  10579464   
4  You are a biomedical relation extraction exper...  No_Relation  10579464   

   run  prompt_id  
0    1          1  
1    1          2  
2    1          3  
3    1          4  
4    1          5  
                                              Prompt     Relation     DocID  \
0  You are a biomedical relation extraction exper...  No_Relation  10579464   
1  You are a biomedical r

In [12]:
print(calibration_test["n_1"]["run_1"]["multi_examples"][0])
print(calibration_test["n_1"]["run_1"]["single_examples"][0])

You are a biomedical relation extraction expert.
Prompt_ID:None
Task: Identify if the realtion between two specified entities is chemically inducing a disease or symptoms of a disease.

Valid relation types (respond with EXACTLY ONE of these words):
- CID: stands for Chemically induced Disease, give this label if the two enteties are a chemical and a disease where the chemical induces the disease or its symptoms. 
- No_Relation: The two enteties are according to the text not involved in chemically inducing a Disease.

CRITICAL INSTRUCTIONS:
1. Only annotate EXPLICIT relations stated in the text
2. If entities co-occur WITHOUT a clear relationship → respond with "No_Relation"
3. Do NOT infer or assume relationships not explicitly stated
4. When uncertain → respond with "No_Relation"
Example 1
## Input:
Text: Intraoperative bradycardia and hypotension associated with timolol and pilocarpine eye drops. A 69-yr-old man, who was concurrently being treated with pilocarpine nitrate and timolo